# EE4211 Project

       Team 1: Quadratic                   Done by Choi Jae Hyung, Lee Min Young, Nicholas Lui Ming Yang, Tran Duy Anh

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from functools import reduce



In [2]:
df = pd.read_csv("project_data.csv")
# df.shape - (1584823, 3)
# df column values:
# localminute - Timestamp 
# dataid - MeterID 
# meter_value - meter reading

# Q1.1

# How many houses are included in the measurement study?

In [3]:
# Since dataid is unique for each meter, we can count the number of unique dataid numbers
df.dataid.value_counts().size

157

### This gives us 157 for the number of houses included in the study

# Are there any malfunctioning meters? If so, identify them and the time periods where they were malfunctioning.

## There are various ways to define a malfunctioning meter. Let us explore some of them

### Let us first check, if there are *meters that report a lower value at a later timestamp*. Since meter_value is cumulative consumption, meter_value should not be lower at a later timestamp for the same meter

In [4]:
grouped = df.groupby(['dataid'], sort=['localminute'])
def has_decreasing_values(df):
    current_value = 0
    for index, val in df.iteritems():
        if val < current_value:
            return True
        current_value = val
        
meters_with_decreasing = (grouped['meter_value']
                          .agg(has_decreasing_values)
                          .where(lambda x: x)
                          .dropna()
                          .keys())

In [5]:
print(len(meters_with_decreasing))
meters_with_decreasing

43


Int64Index([  35,   77,   94,  483,  484, 1042, 1086, 1185, 1507, 1556, 1718,
            1790, 1801, 2129, 2335, 2449, 3134, 3527, 3544, 3893, 4031, 4193,
            4514, 4998, 5129, 5131, 5193, 5403, 5810, 5814, 5892, 6836, 7017,
            7030, 7117, 7739, 7794, 7989, 8156, 8890, 9134, 9639, 9982],
           dtype='int64', name='dataid')

#### Wow, we have 43 meters that have a decreasing value! Let's zoom in and get the offending sections for each meter. (In other words, find the exact data points which show this descreasing value)

In [7]:
## Need to re-sort after filter since filter takes all rows and breaks original sorting
dec_meters = (grouped.filter(lambda x: int(x.name) in meters_with_decreasing)
              .groupby(['dataid'], sort=['localminute']))

## Iterate over values to find offending rows for each meter
## WARNING: RUNS VERY SLOWLY. TODO: OPTIMIZE
offending_values = {}
for group_id, rows in dec_meters.groups.items():
    offending_values[group_id] = []
    current_value = 0
    group_rows = dec_meters.get_group(group_id)
    group_rows_count = group_rows.shape[0]
    for i in range(group_rows_count):
        if group_rows.iloc[i]['meter_value'] < current_value:
            offending_values[group_id].append([group_rows.iloc[i-1], group_rows.iloc[i]])
        current_value = group_rows.iloc[i]['meter_value']
    

KeyboardInterrupt: 

##### Number of 'broken data' instances for each meter

In [ ]:
print("Meter ID |", "Number of broken readings")
for k, v in offending_values.items():
    print(str(k).ljust(20), len(v))
        

##### Just knowing the number of broken readings may not be useful. 
##### Let's say that we want to know which meters should be fixed, since faulty meters result in us inaccurately measuring consumption, and possibly losing money.
##### There should be some measure of tolerance in deciding if a meter is broken. In this case, let's check the average decreasing amount, and the ratio of "broken" readings 

In [ ]:
print("Meter ID |", "Number of broken readings |", "Average decrease across broken readings |", "Percentage of broken readings for meter")
for k, v in offending_values.items():
    print(str(k).ljust(20), 
          str(len(v)).ljust(20), 
          str(reduce(lambda x, y: x + abs(y[1]['meter_value'] - y[0]['meter_value']) / len(v), [0] + v )).ljust(40), 
          (len(v) / dec_meters.get_group(k).shape[0]) * 100)

#### With the data laid out like this, it is pretty clear that most meters are not really broken if we allow a 2% error rate
#### However, in terms of error volume, some are pretty suspect. Let's use an average volume error of 100 Cubic foot (that's a lot!) as our threshold, and filter our results

In [ ]:
print("Meter ID |", "Number of broken readings |", "Average decrease across broken readings |", "Percentage of broken readings for meter")
for k, v in offending_values.items():
    measure = str(reduce(lambda x, y: x + abs(y[1]['meter_value'] - y[0]['meter_value']) / len(v), [0] + v )).ljust(40)
    if float(measure) > 10:
        print(str(k).ljust(20), 
              str(len(v)).ljust(20), 
              measure, 
              (len(v) / dec_meters.get_group(k).shape[0]) * 100)

In [ ]:
"""Use this function to validate/check each bad meter given our heuristic"""
def print_bad_meter_readings(meterID):
    meter_readings = offending_values[meterID]
    print("time apart |".ljust(20), "meter value initial |", "meter value after |", "difference")
    for readings_pair in meter_readings:

        print(str(pd.to_datetime(readings_pair[1]['localminute']) - pd.to_datetime(readings_pair[0]['localminute'])).ljust(25), 
              str(readings_pair[0]['meter_value']).ljust(20), 
              str(readings_pair[1]['meter_value']).ljust(20), 
              readings_pair[1]['meter_value'] - readings_pair[0]['meter_value'])
print_bad_meter_readings(1556)
        

### Another requirement of the meters is that they push their readings to be saved if the meter values differ by at least 2 cubic foot
#### Let us verify that every pair of readings for a meter differ by at least 2 cubic foot. To not double count, we will only check for readings where the differing value is 0 >= x > 2 so that we don't get the same error readings where the readings decrease
#### Since the value is smaller, we will not focus on the difference in values, but on the percentage of total readings for the meter only

In [ ]:
grouped2 = df.groupby(['dataid'], sort=['localminute'])
def has_stagnant_values(df):
    current_value = 0
    for index, val in df.iteritems():
        if index == 0:
            current_value = val
            continue
        
        if val < current_value + 2 and val >= current_value:
            return True
        current_value = val
        
meters_with_stagnant = (grouped['meter_value']
                          .agg(has_stagnant_values)
                          .where(lambda x: x)
                          .dropna()
                          .keys())

print("Number of meters with stagnant values: ", len(meters_with_stagnant))

#### The following segment should have calculated the percentage of stagnant values for each meter, but it takes too long (tried for 5 minutes and gave up). At least we know number of meters that reported stagnant values?

In [ ]:
# ## Need to re-sort after filter since filter takes all rows and breaks original sorting
# stagnant_meters = (grouped2.filter(lambda x: int(x.name) in meters_with_stagnant)
#               .groupby(['dataid'], sort=['localminute']))

# ## Iterate over values to count offending occurences. Not counting value
# ## WARNING: RUNS VERY SLOWLY. TODO: OPTIMIZE
# offending_values2 = {}
# for group_id, rows in stagnant_meters.groups.items():
#     offending_values2[group_id] = 0
#     group_rows = stagnant_meters.get_group(group_id)
#     group_rows_count = group_rows.shape[0]
#     # Set current_value so we do not trigger first reading
#     current_value = group_rows.iloc[0]['meter_value'] - 5
#     for i in range(group_rows_count):
#         if group_rows.iloc[i]['meter_value'] < current_value + 2 and group_rows.iloc[i]['meter_value'] >= current_value:
#             offending_values2[group_id] += 1
#         current_value = group_rows.iloc[i]['meter_value']

        
# print("Meter ID |", "Number of broken readings |", "Percentage of broken readings for meter")
# for k, v in offending_values2.items():
#     print(str(k).ljust(20), 
#           str(v).ljust(20),  
#           (v / stagnant_meters.get_group(k).shape[0]) * 100)


In [ ]:
stagnant_meters = (grouped2.filter(lambda x: int(x.name) in meters_with_stagnant)
              .groupby(['dataid'], sort=['localminute']))

print("Total number of readings to iterate, which is why it is slow: ", 
      reduce(lambda x, y: x + len(y), [0] + list(stagnant_meters.groups.values())))

### We now have two different criterion for deciding what constitues a broken meter, and the readings that helped us determine that. Since the question asks for "time periods where they were malfunctioning.", let's demonstrate that we can consolidate broken readings into time periods

In [ ]:
# let's use the meter with the most broken readings (based off decreasing values)
most_broken_values_meter = grouped.get_group(5403)

In [ ]:
# We are using a stricter requirement now. Let's consider that, if the subsequent meter reading is less 
# than an increment of 2 (meters are only supposed to report after at least an increment of 2)

# Broken_criteria is a helper function (comparator function) that takes in two (in-order) readings and outputs a 
# boolean of whether the later reading is "broken"

broken_criteria = lambda x, y: y['meter_value'] < x['meter_value'] + 2

In [ ]:
broken_readings = 0
num_readings = most_broken_values_meter.shape[0]
for i in range(1, num_readings):
    if broken_criteria(most_broken_values_meter.iloc[i-1], most_broken_values_meter.iloc[i]):
        broken_readings += 1

print("Number of broken readings for meter 5403, based on stricter requirements: ", broken_readings)

In [ ]:
# Let's now create a function that can aggregate a meter's readings into "broken" time periods
# Return value of the function will be as follow:
"""
2D array. 
First dimension is the time periods. 
Second dimension is the consecutive broken readings that make up the broken time period
To find the actual time data, take the ['localminute'] attribute from the first and last reading per period
    time_periods = [
        [reading_1, reading_2, reading_3],
        [reading_1, reading_2]
    ]
"""
def get_broken_time_periods(broken_criteria, meter_readings):
    num_readings = meter_readings.shape[0]
    time_periods = []
    temp_period = []
    for i in range(1, num_readings):
        if broken_criteria(meter_readings.iloc[i-1], meter_readings.iloc[i]):
            temp_period.append(meter_readings.iloc[i])
        else:
            if temp_period:
                time_periods.append(temp_period)
                temp_period = []
    if temp_period:
        time_periods.append(temp_period)
    return time_periods

In [ ]:
broken_time_periods = get_broken_time_periods(broken_criteria, most_broken_values_meter)
print("Number of broken time periods: ", len(broken_time_periods))


# Q1.2 
## Hourly Data Collection and Plotting the Data

#### Objective: 
Obtain hourly data from a list of given data

#### Parameters: 
#### 1. Gas meter ID 
Gas meter with ID 739 will be focused for the study
#### 2. Starting time
1st Oct 2015, 6 a.m.
#### 3. Ending time
1st Nov 2015, 6 a.m.

#### Assumption:
1. Starting time is within the given data range.
2. The data is cumulative: either constant or increasing. Hence, the hourly data is obtained from the most recent data from the previous hour.

#### How to handle missing data?
When there is a period of missing data, there may not be most recent data present close to an exact hour.
As the data is cumulative gas consumption by household, when there is no data, it can be assumed as there is insignificant gas consumption between that period.

Hence, the missing data will be given same value as most recent hourly data obtained.

## Code

#### The algorithm of hourly data collection consists of the functions below.

1. When the hour hand changes, collect hourly data from the previous data.

2. Look out for missing hour. When missing hour is found, previuos hour data is given for that hour.

3. When starting hour is exact hour (hh:00:00), hourly data begins to be collected from next hour.

When a list of data is provided, obtaining hourly data from the starting exact hour may lead to over-estimation as there could possibly be a lot of consumption between the exact hour and the next data found after that hour.

When ending hour is hh:00:00, hourly data of hh:00:00 will be the last hourly data.

In [8]:
# Hourly data collected
hourly_meter = []

#period_start format: 'yyyy-mm-dd hh:mm:ss'
def start_period(period_start):
    start_datetime = pd.to_datetime(period_start)
    return start_datetime

#period_end format: 'yyyy-mm-dd hh:mm:ss'
def end_period(period_end):
    end_datetime = pd.to_datetime(period_end)
    return end_datetime                
    
def hourly_reading():
       
    hourly_meter.clear()
    first_hour = t.dt.hour[0]
    
    for dataframe_row, value in t.dt.hour.iteritems():
        
        if (dataframe_row != 0):
            
            # Smallest difference is 28 days in Feb - First day of the month. Hence 28 - 1 = 27
            if ((t[dataframe_row].day - t[dataframe_row - 1].day) >= 27):
                print ("There is missing data while the month is changed. Please check.")
                break

            else:
                if (t.dt.hour[dataframe_row] != 0):
                    if (t[dataframe_row].day != t[dataframe_row - 1].day):
                        if ((t[dataframe_row].hour - t[dataframe_row - 1].hour) >= 0):
                            for i in range (0, 24*(t[dataframe_row].day - t[dataframe_row - 1].day)):
                                hourly_meter.append(temp.meter_value[dataframe_row-1])

                        else:
                            # e.g. final hour is day 2 00:00:00, initial hour is day 1 20:00:00,  there is 4 hours gap, not 28 hours.
                            if ((t[dataframe_row].day - t[dataframe_row - 1].day) > 1):
                                for i in range (0, 24*(t[dataframe_row].day - t[dataframe_row - 1].day - 1)):
                                    hourly_meter.append(temp.meter_value[dataframe_row-1])

                            else:
                                return
                
        if (t.dt.hour[dataframe_row] == first_hour):
                        
            # Taking note of jump in data 
            # If there is data that is hh:00:00 and this is starting point, data will be added as the first hourly value.
            # Else, the first hourly value will be the next hour.
            if (dataframe_row == 0):
                if ((t[0].minute == 0) and (t[0].second == 0)):
                    hourly_meter.append(temp.meter_value[0])
            
        else:
            #One data prior to the time going beyond a new hour hand is assumed to be the meter value when the new hour strikes.
            hourly_meter.append(temp.meter_value[dataframe_row-1]) 
            
            first_hour = first_hour + 1
            
            if (first_hour == 24):
                first_hour = 0
            
            hourly_missing_data(dataframe_row, first_hour)

    # The last hour of the ending point will be considered even if there is no data after that hh:00:00
    if (t.dt.hour[len(t)-1] != end_datetime.hour):
        if ((end_datetime.hour - t.dt.hour[len(t)-1]) < 0):
            for i in range (0, 24 + (end_datetime.hour - t.dt.hour[len(t)-1])):
                hourly_meter.append(temp.meter_value[len(t)-1])
        else:
            for i in range (0, end_datetime.hour - t.dt.hour[len(t)-1]):
                hourly_meter.append(temp.meter_value[len(t)-1])
           
    if (t.dt.day[len(t)-1] != end_datetime.day):
        for i in range (0, 24 * (end_datetime.day - t.dt.day[len(t)-1])):                            
            hourly_meter.append(temp.meter_value[len(t)-1])

            
def hourly_missing_data(dataframe_row, first_hour):
    if (t.dt.hour[dataframe_row] != first_hour):
        print ("There is missing data for an hour...................................................................")
        first_hour = first_hour + 1
        hourly_meter.append(temp.meter_value[dataframe_row-1])
    
        if (first_hour == 24):
            first_hour = 0
    
    # Repeat if there is missing data for more than an hour
    if (t.dt.hour[dataframe_row] != first_hour):
        return hourly_missing_data(dataframe_row, first_hour)
    return 

#### Choose the meter_ID, starting time and ending time of the hourly data collection.

In [9]:
#Format: ID No.
ID = 739

#Format: 'yyyy-mm-dd hh:mm:ss'
start_datetime = start_period('2015-10-01 06:00:00')

#Format: 'yyyy-mm-dd hh:mm:ss'
end_datetime = end_period('2015-11-01 06:00:00')

#### Collect hourly data.

In [10]:
grouped = df.groupby(['dataid'], sort=['localminute'])
temp = grouped.get_group(ID).copy()
temp['datetime'] = pd.to_datetime(temp['localminute'])
temp = temp.loc[(temp['datetime'] >= start_datetime) & (temp['datetime'] <= end_datetime)]
t = pd.to_datetime(temp['datetime'])
temp.index = range(temp.shape[0])
t.index = range(t.shape[0])


hourly_reading()

### Plot the hourly reading.

The vertical limit of the graph is gauged by the lowest cumulative value of the gas consumption to the highest.

In [11]:
ylim_low = hourly_meter[0]
ylim_high = hourly_meter[len(hourly_meter)-1]
print ("The plot has vertical range from", ylim_low, "to", ylim_high)

The plot has vertical range from 88858 to 89544


In [12]:
x = np.arange(len(hourly_meter))

fig, ax = plt.subplots(1, 1, figsize=(20, 6))
ax.plot(x, hourly_meter, 'ok', ms=1)
ax.set_xlim(0, len(hourly_meter) + 5)
ax.set_ylim(ylim_low - 10, ylim_high + 10)
ax.set_title('Generative model')
plt.xlabel ("Hours")
plt.ylabel ("Meter value")

## Q1.3 Find for each home, 5 houses with highest correlation

In [3]:
#get a list of meter id
idList = df['dataid'].unique()

df["localminute"] = pd.to_datetime(df["localminute"])
#get a dataframe of data by hour:
df_byhour = pd.DataFrame(df.groupby([df["localminute"].dt.date.rename("Date"), 
                               df["localminute"].dt.hour.rename("Hour"), 
                               df["dataid"]]).mean().reset_index())

#add a column called "DateHour" in the dataframe:
df_byhour['DateHour'] = pd.to_datetime(df_byhour['Date'].apply(str)+' '+df_byhour['Hour'].apply(str)+':00:00')

In [4]:
df_byhour

,Date,Hour,dataid,meter_value,DateHour
0,2015-10-01,5,35,93470.0,2015-10-01 05:00:00
1,2015-10-01,5,94,116642.0,2015-10-01 05:00:00
2,2015-10-01,5,114,128294.0,2015-10-01 05:00:00
3,2015-10-01,5,187,263272.0,2015-10-01 05:00:00
4,2015-10-01,5,222,612262.0,2015-10-01 05:00:00
5,2015-10-01,5,252,329214.0,2015-10-01 05:00:00
6,2015-10-01,5,370,87880.0,2015-10-01 05:00:00
7,2015-10-01,5,483,360456.0,2015-10-01 05:00:00
8,2015-10-01,5,484,99298.0,2015-10-01 05:00:00
9,2015-10-01,5,739,88858.0,2015-10-01 05:00:00


In [9]:
#there are 183 days => 4392 hours => 4392 readings in each list
hourlyDataDict = {}

staHr = pd.Timestamp('2015-10-01 05:00:00')
endHr = pd.Timestamp('2016-04-01 04:00:00')
for id1 in idList:
    prevHr = staHr
    dfTemp = df_byhour.groupby(['dataid']).get_group(id1)
    hourList = dfTemp['DateHour'].tolist()
    valList = dfTemp['meter_value'].tolist()
    
    prevVal = valList[0]
    newList = [valList[0]]
    for index, hr in enumerate(hourList):
        #if hour is more than an hour bigger than previous
        while (hr - prevHr).seconds >= 3600:
            newList.append(prevVal)
            #increment every hour
            prevHr += pd.Timedelta(seconds=3600)
            #print("append in loop" + " hour is: " + str(prevHr))
    

        prevVal = valList[index]
    
    while prevHr < endHr:
        newList.append(prevVal)
        #increment every hour
        prevHr += pd.Timedelta(seconds=3600)
    
    hourlyDataDict[id1] = newList



In [10]:
print(len(hourlyDataDict[739]))
print(len(hourlyDataDict[35]))

4392
4392


In [11]:
corDict = {}
for id1 in idList:
    corDict[id1] = {}
    for id2 in idList:
        coef = np.corrcoef(hourlyDataDict[id1], hourlyDataDict[id2])[0, 1]
        corDict[id1][id2] = coef
        #print("coefficient between " + str(id1) + " and " + str(id2) + " is: " + str(coef))

coefficient between 739 and 739 is: 1.0
coefficient between 739 and 8890 is: 0.999537795736
coefficient between 739 and 6910 is: 0.999131361131
coefficient between 739 and 3635 is: 0.999108266936
coefficient between 739 and 1507 is: 0.999387339247
coefficient between 739 and 5810 is: 0.998416152431
coefficient between 739 and 484 is: 0.995951012115
coefficient between 739 and 4352 is: 0.936169381291
coefficient between 739 and 1718 is: 0.998969092177
coefficient between 739 and 1714 is: 0.9973515561
coefficient between 739 and 9849 is: 0.992377201799
coefficient between 739 and 5131 is: 0.998836162822
coefficient between 739 and 6412 is: 0.998965486863
coefficient between 739 and 7429 is: 0.998951833273
coefficient between 739 and 871 is: 0.998781365259
coefficient between 739 and 1086 is: 0.999410090509
coefficient between 739 and 1589 is: 0.996636146959
coefficient between 739 and 8156 is: 0.994444252207
coefficient between 739 and 9631 is: 0.995241155099
coefficient between 739 and 

coefficient between 8890 and 5892 is: 0.998810490332
coefficient between 8890 and 7900 is: 0.999358705421
coefficient between 8890 and 8084 is: 0.998089394998
coefficient between 8890 and 3527 is: 0.998341193486
coefficient between 8890 and 3849 is: 0.988333104926
coefficient between 8890 and 4356 is: 0.998904657145
coefficient between 8890 and 2129 is: 0.998039046075
coefficient between 8890 and 2965 is: 0.989507392158
coefficient between 8890 and 2575 is: 0.997879644129
coefficient between 8890 and 8086 is: 0.997110722488
coefficient between 8890 and 1415 is: 0.961895687757
coefficient between 8890 and 2233 is: 0.933768271093
coefficient between 8890 and 6863 is: 0.568193154157
coefficient between 8890 and 2470 is: 0.955420065541
coefficient between 8890 and 3778 is: 0.998603428528
coefficient between 8890 and 2378 is: 0.990442464305
coefficient between 8890 and 6830 is: 0.996756057278
coefficient between 8890 and 9766 is: 0.952046876933
coefficient between 8890 and 4193 is: 0.982230

coefficient between 3635 and 3527 is: 0.996374629926
coefficient between 3635 and 3849 is: 0.991354908857
coefficient between 3635 and 4356 is: 0.999168407177
coefficient between 3635 and 2129 is: 0.998985065592
coefficient between 3635 and 2965 is: 0.991928257135
coefficient between 3635 and 2575 is: 0.998129196684
coefficient between 3635 and 8086 is: 0.997888813686
coefficient between 3635 and 1415 is: 0.96710799237
coefficient between 3635 and 2233 is: 0.940892862371
coefficient between 3635 and 6863 is: 0.578860239819
coefficient between 3635 and 2470 is: 0.961227194727
coefficient between 3635 and 3778 is: 0.999149982203
coefficient between 3635 and 2378 is: 0.992603979334
coefficient between 3635 and 6830 is: 0.994599696486
coefficient between 3635 and 9766 is: 0.9589923921
coefficient between 3635 and 4193 is: 0.985832012666
coefficient between 3635 and 8703 is: 0.581189430307
coefficient between 3635 and 6578 is: 0.803845998606
coefficient between 3635 and 4228 is: 0.938871959

coefficient between 5810 and 9134 is: 0.983324504601
coefficient between 5810 and 4767 is: 0.99589484056
coefficient between 5810 and 5785 is: 0.995272278261
coefficient between 5810 and 7739 is: 0.984988833805
coefficient between 5810 and 7117 is: 0.988192518851
coefficient between 5810 and 5193 is: 0.996186576185
coefficient between 5810 and 3367 is: 0.996563346795
coefficient between 5810 and 8467 is: 0.95171707207
coefficient between 5810 and 5972 is: 0.998182231528
coefficient between 5810 and 5439 is: 0.995316228345
coefficient between 5810 and 114 is: 0.998060402306
coefficient between 5810 and 9474 is: 0.941273493837
coefficient between 5810 and 2335 is: 0.998269296521
coefficient between 5810 and 7460 is: 0.947118410105
coefficient between 5810 and 4998 is: 0.99649103617
coefficient between 5810 and 9278 is: 0.991681735592
coefficient between 5810 and 5129 is: 0.962391791122
coefficient between 5810 and 2018 is: 0.998584103007
coefficient between 5810 and 9052 is: 0.9959919615

coefficient between 4352 and 2449 is: 0.880200827237
coefficient between 4352 and 35 is: 0.921837343406
coefficient between 4352 and 1697 is: 0.916812371319
coefficient between 4352 and 9134 is: 0.907399238865
coefficient between 4352 and 4767 is: 0.927624034032
coefficient between 4352 and 5785 is: 0.945518024293
coefficient between 4352 and 7739 is: 0.893876071745
coefficient between 4352 and 7117 is: 0.898131481542
coefficient between 4352 and 5193 is: 0.914730171488
coefficient between 4352 and 3367 is: 0.921952232054
coefficient between 4352 and 8467 is: 0.98347288436
coefficient between 4352 and 5972 is: 0.938533924312
coefficient between 4352 and 5439 is: 0.92750692
coefficient between 4352 and 114 is: 0.936931501689
coefficient between 4352 and 9474 is: 0.99494585671
coefficient between 4352 and 2335 is: 0.922811664603
coefficient between 4352 and 7460 is: 0.994540878689
coefficient between 4352 and 4998 is: 0.925181505599
coefficient between 4352 and 9278 is: 0.957868503147
co

...

coefficient between 2946 and 1283 is: 0.300666067599
coefficient between 2946 and 1792 is: 0.28793034518
coefficient between 2946 and 4031 is: 0.275632343325
coefficient between 2946 and 3893 is: 0.283720463542
coefficient between 2946 and 1185 is: 0.286329334647
coefficient between 2946 and 7682 is: 0.308597250103
coefficient between 2946 and 2638 is: 0.286834806342
coefficient between 2946 and 187 is: 0.378439671651
coefficient between 2946 and 3723 is: 0.290100135907
coefficient between 2946 and 4514 is: 0.287612442476
coefficient between 2946 and 5814 is: 0.280083972139
coefficient between 2946 and 77 is: 0.334529363499
coefficient between 2946 and 1619 is: 0.285743778603
coefficient between 2946 and 4421 is: 0.296998081224
coefficient between 2946 and 661 is: 0.283665084528
coefficient between 2946 and 3134 is: 0.285729058276
coefficient between 2946 and 6836 is: 0.272656267661
coefficient between 2946 and 1042 is: 0.333301997602
coefficient between 2946 and 4373 is: 0.34195142137

coefficient between 1403 and 2980 is: 0.332449821001
coefficient between 1403 and 2094 is: 0.343521076092
coefficient between 1403 and 5892 is: 0.356884028505
coefficient between 1403 and 7900 is: 0.321692656102
coefficient between 1403 and 8084 is: 0.343727140134
coefficient between 1403 and 3527 is: 0.318467271155
coefficient between 1403 and 3849 is: 0.377247468235
coefficient between 1403 and 4356 is: 0.34855309421
coefficient between 1403 and 2129 is: 0.353891323555
coefficient between 1403 and 2965 is: 0.385996931442
coefficient between 1403 and 2575 is: 0.346048360847
coefficient between 1403 and 8086 is: 0.365026237374
coefficient between 1403 and 1415 is: 0.391360314766
coefficient between 1403 and 2233 is: 0.357703802997
coefficient between 1403 and 6863 is: 0.674453449534
coefficient between 1403 and 2470 is: 0.423394557585
coefficient between 1403 and 3778 is: 0.349531547486
coefficient between 1403 and 2378 is: 0.354050831497
coefficient between 1403 and 6830 is: 0.3225250

coefficient between 6673 and 5814 is: 0.721542626664
coefficient between 6673 and 77 is: 0.766830855721
coefficient between 6673 and 1619 is: 0.728728087006
coefficient between 6673 and 4421 is: 0.76992152874
coefficient between 6673 and 661 is: 0.733404155619
coefficient between 6673 and 3134 is: 0.735345655471
coefficient between 6673 and 6836 is: 0.688176858017
coefficient between 6673 and 1042 is: 0.800275229498
coefficient between 6673 and 4373 is: 0.841677354775
coefficient between 6673 and 5484 is: 0.721196922017
coefficient between 6673 and 2072 is: 0.730670835089
coefficient between 6673 and 2461 is: 0.684302784055
coefficient between 6673 and 744 is: 0.68828627001
coefficient between 6673 and 9982 is: 0.730056016157
coefficient between 6673 and 1790 is: 0.731273765411
coefficient between 6673 and 1800 is: 0.687827998979
coefficient between 6673 and 2980 is: 0.686839583752
coefficient between 6673 and 2094 is: 0.719782966947
coefficient between 6673 and 5892 is: 0.733679524032

coefficient between 2814 and 3527 is: 0.284897506119
coefficient between 2814 and 3849 is: 0.337265685076
coefficient between 2814 and 4356 is: 0.312030692028
coefficient between 2814 and 2129 is: 0.316987252786
coefficient between 2814 and 2965 is: 0.347879012012
coefficient between 2814 and 2575 is: 0.309476982971
coefficient between 2814 and 8086 is: 0.327640373424
coefficient between 2814 and 1415 is: 0.350709231509
coefficient between 2814 and 2233 is: 0.319854938483
coefficient between 2814 and 6863 is: 0.602712500795
coefficient between 2814 and 2470 is: 0.378764701573
coefficient between 2814 and 3778 is: 0.313345232098
coefficient between 2814 and 2378 is: 0.316330253776
coefficient between 2814 and 6830 is: 0.28880855073
coefficient between 2814 and 9766 is: 0.377205924445
coefficient between 2814 and 4193 is: 0.318863874927
coefficient between 2814 and 8703 is: 0.640023240544
coefficient between 2814 and 6578 is: 0.4721363845
coefficient between 2814 and 4228 is: 0.414388041

coefficient between 4874 and 5814 is: 0.116725437741
coefficient between 4874 and 77 is: 0.141128130701
coefficient between 4874 and 1619 is: 0.119336707848
coefficient between 4874 and 4421 is: 0.12380980318
coefficient between 4874 and 661 is: 0.118035563723
coefficient between 4874 and 3134 is: 0.119210851561
coefficient between 4874 and 6836 is: 0.113695965275
coefficient between 4874 and 1042 is: 0.139844391347
coefficient between 4874 and 4373 is: 0.142029097369
coefficient between 4874 and 5484 is: 0.121860938954
coefficient between 4874 and 2072 is: 0.119226202181
coefficient between 4874 and 2461 is: 0.107361100197
coefficient between 4874 and 744 is: 0.110878202874
coefficient between 4874 and 9982 is: 0.125894433409
coefficient between 4874 and 1790 is: 0.116115712111
coefficient between 4874 and 1800 is: 0.116069765695
coefficient between 4874 and 2980 is: 0.115694671967
coefficient between 4874 and 2094 is: 0.118629199409
coefficient between 4874 and 5892 is: 0.12404574317

In [12]:
print(corDict)

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-12-5fb2c98b8154>, line 1)